In [51]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime as dt
from datetime import timedelta
import os
import json
import csv

In [16]:
now_bangkok = (dt.now() + timedelta(hours=7)).isoformat()[:19]
ytd_bangkok = (dt.now() + timedelta(hours=7, days=-30)).isoformat()[:19]

now_YYYY = now_bangkok[:4]
now_mm   = now_bangkok[5:7]
now_dd   = now_bangkok[8:10]

ytd_YYYY = ytd_bangkok[:4]
ytd_mm   = ytd_bangkok[5:7]
ytd_dd   = ytd_bangkok[8:10]

In [17]:
stock = 'ADVANC'
url   = f"https://www.set.or.th/th/market/product/stock/quote/{stock}/news"
url2  = f"https://www.set.or.th/api/set/news/search?symbol={stock}&fromDate={ytd_dd}%2F{ytd_mm}%2F{ytd_YYYY}&toDate={now_dd}%2F{now_mm}%2F{now_YYYY}&keyword=&lang=th"

In [18]:
session = requests.Session()
session.get(url)
session

In [19]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='charlot', value='b2cd1c28-9d6d-4120-af04-55ded12ac0fe', port=None, port_specified=False, domain='www.set.or.th', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1747413250, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='visid_incap_2046605', value='/28mMiv3SfCmaxklkpTTGoIXJmgAAAAAQUIPAAAAAABO0+mpt9XwEqSxS0ag9Tn8', port=None, port_specified=False, domain='.set.or.th', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1778859270, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='nlbi_2046605', value='VVt2EFIHfBaqcE4DydyeTAAAAABTvitXY626cdszxi1sCIgf', port=None, port_specified=False, domain='.set.or.th', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=No

In [20]:
res = session.get(
    url2,
    headers={
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'priority': 'u=1, i',
        'referer': f'https://www.set.or.th/th/market/product/stock/quote/{stock}/news',
    }
)
res_json = res.json()
len(res_json)

2

In [21]:
res_json

{'totalCount': 5,
 'newsInfoList': [{'id': '96155401',
   'datetime': '2025-05-06T19:05:00+07:00',
   'symbol': 'ADVANC',
   'source': 'ADVANC',
   'url': 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=96155401&symbol=ADVANC',
   'headline': 'คำอธิบายและวิเคราะห์ของฝ่ายจัดการ ไตรมาสที่ 1 สิ้นสุดวันที่ 31 มี.ค. 2568 (แก้ไข ARPU)',
   'isTodayNews': False,
   'viewClarification': None,
   'marketAlertTypeId': None,
   'tag': '',
   'percentPriceChange': None,
   'product': 'S',
   'lang': 'th'},
  {'id': '96147401',
   'datetime': '2025-05-06T17:53:00+07:00',
   'symbol': 'ADVANC',
   'source': 'ADVANC',
   'url': 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=96147401&symbol=ADVANC',
   'headline': 'การแต่งตั้งกรรมการแทนกรรมการที่ลาออก และการเปลี่ยนแปลงกรรมการในคณะกรรมการชุดย่อย',
   'isTodayNews': False,
   'viewClarification': None,
   'marketAlertTypeId': None,
   'tag': '',
   'percentPriceChange': None,
   'product': 'S',
   'lang': 'th'},
  {'id': '

In [50]:
daily_dir_path_1 = f'../data/1_raw_response/01_set/{now_YYYY}-{now_mm}-{now_dd}'
daily_dir_path_2 = f'../data/2_clean_response/01_set/{now_YYYY}-{now_mm}-{now_dd}'
daily_dir_path_3 = f'../data/3_raw_news/01_set/{now_YYYY}-{now_mm}-{now_dd}'

if not os.path.isdir(daily_dir_path_1):
    os.makedirs(daily_dir_path_1)
    
if not os.path.isdir(daily_dir_path_2):
    os.makedirs(daily_dir_path_2)
    
if not os.path.isdir(daily_dir_path_3):
    os.makedirs(daily_dir_path_3)

In [52]:
for i_news, a_news in enumerate(res_json['newsInfoList']):
    try:
        res = requests.get(a_news['url'])
        print(f'Success {i_news:05d}.')
    except:
        print(f'Fail to fetch url `{a_news['url']}`.')

    with open(f'{daily_dir_path_1}/{i_news:05d}.json', 'w') as json_file:
        json.dump(
            {
                "original_req": a_news,
                "status_code": res.status_code,
                "text": res.text,
            }, 
            json_file
        )

Success 00000.
Success 00001.
Success 00002.
Success 00003.
Success 00004.


# Scrape news title, news content

In [ ]:
raw_news_data = []
raw_news_fields = ['news_datetime', 'news_title', 'news_content']

for i_news, a_news in enumerate(res_json['newsInfoList']):
    with open(f'{daily_dir_path_1}/{i_news:05d}.json', 'r') as json_file:
        try:
            res = json.load(json_file)
            res_text = res['text']

            # print(res_text)

            soup = BeautifulSoup(res_text)

            news_datetime = res['original_req']['datetime']
            news_title    = soup.title.string
            news_content  = soup.select('.raw-html-new')[0].select(".raw-html")[0].string

            a_news = {
                "news_datetime": news_datetime,
                "news_title":    news_title,
                "news_content":  news_content,
            }

            raw_news_data.append(a_news)

            with open(f'{daily_dir_path_2}/{i_news:05d}.json', 'w') as json_file:
                json.dump(a_news, json_file)
            
            print(f'Success {i_news:05d}. {news_title}')
        except:
            print(f'Fail reading raw json `{a_news['url']}`.')

with open(f'{daily_dir_path_3}/{i_news:05d}.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=raw_news_fields)
    writer.writeheader()
    writer.writerows(raw_news_data)

Success 00000. คำอธิบายและวิเคราะห์ของฝ่ายจัดการ ไตรมาสที่ 1 สิ้นสุดวันที่ 31 มี.ค. 2568 (แก้ไข ARPU) - ตลาดหลักทรัพย์แห่งประเทศไทย
Success 00001. การแต่งตั้งกรรมการแทนกรรมการที่ลาออก และการเปลี่ยนแปลงกรรมการในคณะกรรมการชุดย่อย - ตลาดหลักทรัพย์แห่งประเทศไทย
Success 00002. งบการเงิน ไตรมาสที่ 1/2568 (สอบทานแล้ว) - ตลาดหลักทรัพย์แห่งประเทศไทย
Success 00003. คำอธิบายและวิเคราะห์ของฝ่ายจัดการ ไตรมาสที่ 1 สิ้นสุดวันที่ 31 มี.ค. 2568 - ตลาดหลักทรัพย์แห่งประเทศไทย
Success 00004. สรุปผลการดำเนินงานของ บจ. ไตรมาสที่ 1 (F45) (สอบทานแล้ว) - ตลาดหลักทรัพย์แห่งประเทศไทย
